In [0]:
class DeltaShareProvider:
  def __init__(self, share:str, recipient:str, recipient_sharing_identifier:str="", drop_share_if_exists:bool=False, drop_recipient_if_exists:bool=False):
    """
    Initializes a DeltaShareProvider instance with the given parameters.

    Args:
        share (str): The name of the Delta share to use.
        recipient (str): The name of the recipient to share the Delta share with.
        recipient_sharing_identifier (str): The Databricks ID of the recipient. Defaults to an empty string.
        drop_if_exists (bool): Whether to drop the recipient and the share if they already exist. Defaults to False.
    """
    self.share = share
    if drop_share_if_exists:
      self.drop_share()
    if drop_recipient_if_exists:
      self.drop_recipient(recipient)
    self.__spark_sql(f"CREATE SHARE IF NOT EXISTS {share};")
    self.add_recipient(recipient, recipient_sharing_identifier)
    
  def drop_share(self):
    """
    Drops the Delta share if it exists.
    """
    self.__log(f"share {self.share} will be dropped.")
    self.__spark_sql(f"DROP SHARE IF EXISTS {self.share};")
  
  def share_catalog(self, catalog:str, enable_cdf:bool=False):
    """
    Shares all databases in the specified catalog to the Delta share.

    Args:
        catalog (str): The name of the catalog to share.
        enable_cdf (bool): Whether to enable change data feed (CDF) on the shared tables. Defaults to False.

    Returns:
        DeltaShareProvider: The instance of DeltaShareProvider.
    """
    databases = self.__get_database_objects("databases", catalog, 'databaseName')
    self.__log(f'sharing all databases in catalog {catalog} to share {self.share}')
    for database in databases:
      if database == "information_schema" or databases == "default":
        self.__log(f"skipping sharing database {catalog}.{database}")
        continue
      self.share_database(f"{catalog}.{database}", enable_cdf)
    self.__log(f'all databases in catalog {catalog} shared in share {self.share}')
    return self
  
  def unshare_catalog(self, catalog:str):
    """
    Removes all databases in the specified catalog from the Delta share.

    Args:
        catalog (str): The name of the catalog to remove from the Delta share.

    Returns:
        DeltaShareProvider: The instance of DeltaShareProvider.
    """
    databases = self.__get_database_objects("databases", catalog, 'databaseName')
    self.__log(f'unsharing all databases in catalog {catalog} from share {self.share}')
    for database in databases:
      if database == "information_schema" or databases == "default":
        continue
      self.unshare_database(f"{catalog}.{database}")
    self.__log(f'catalog {catalog} compeletly unshared from share {self.share}')
    return self
  
  def share_database(self, database:str, enable_cdf:bool=False):
    """
    Shares all tables in the specified database to the Delta share.

    Args:
        database (str): The name of the database to share.
        enable_cdf (bool): Whether to enable change data feed (CDF) on the shared tables. Defaults to False.

    Returns:
        DeltaShareProvider: The instance of DeltaShareProvider.
    """
    tables = self.__get_database_objects("tables", database, 'tableName')
    self.__log(f'sharing all tables in database {database} to share {self.share}')
    for table in tables:
      self.share_table(f"{database}.{table}", enable_cdf)
    self.__log(f'all tables in database {database} shared in share {self.share}')
    return self
  
  def unshare_database(self, database:str):
    """
    Unshares all tables in the specified database from the share.
    
    Args:
        database (str): Name of the database.
    
    Returns:
        self
    """
    tables = self.__get_database_objects("tables", database, 'tableName')
    self.__log(f'unsharing all tables in database {database} from share {self.share}')
    for table in tables:
      self.unshare_table(f"{database}.{table}")
    self.__log(f'database {database} compeletly unshared from share {self.share}')
    return self
  
  def share_table(self, table:str, enable_cdf:bool=False):
    """
    Shares the specified table to the share.
    
    Args:
        table (str): Name of the table.
        enable_cdf (bool): Whether to enable Change Data Feed for the shared table (default False).
    
    Returns:
        self
    """
    try:
      if enable_cdf:
        self.unshare_table(table) #unshare it first to enable cdf
        self.__log(f"enabling CDF on table {table}, this might take few minutes to complete.")
        self.__spark_sql(f'ALTER TABLE {table} SET TBLPROPERTIES (delta.enableChangeDataFeed = true);')
        self.__spark_sql(f'ALTER SHARE {self.share} ADD TABLE {table} WITH HISTORY;')
        self.__log(f'table {table} added to share {self.share} with CDF and History turned on')
      else:
        self.__spark_sql(f'ALTER SHARE {self.share} ADD TABLE {table};')
        self.__log(f'table {table} added to share {self.share}')
    except Exception as e:
      self.__log("exception occured, message is: " + str(e))
    return self
  
  def unshare_table(self, table:str):
    """
    Unshares the specified table from the share.
    
    Args:
        table (str): Name of the table.
    
    Returns:
        self
    """
    try:
      self.__log(f"will try to remove table {table} form share {self.share} first.")
      self.__spark_sql(f'ALTER SHARE {self.share} REMOVE TABLE {table};')
    except Exception as e:
      pass #self.__log("exception occured, message is: " + str(e))
    return self
  
  def add_recipient(self, recipient:str, recipient_sharing_identifier:str=""):
    """
    Adds a recipient to the share and grants SELECT access to them.
    
    Args:
        recipient (str): Name of the recipient.
        recipient_sharing_identifier (str): ID of the Databricks instance where the recipient is located (optional).
    
    Returns:
        self
    """
    if recipient_sharing_identifier is None or recipient_sharing_identifier.strip()=="":
      self.__log(f'recipient {recipient} will be created. copy the code that will show after completion and share it with the recipients.')
      result = self.__spark_sql(f'create recipient if not exists {recipient};').select("info_value").where("info_name='activation_link'").collect()
      if len(result) > 0:
        activation_link = result[0][0]
      else:
        activation_link = "<share_profile_file_location>"
      display(self.__spark_sql(f"select 'DeltaShareRecipient(share_profile_file_loc=\"{activation_link}\", catalog=\"hive_metastore\").create_remotely_linked_tables(share=\"{self.share}\")' as run_this_code_at_recipient"))
    else:
      existing = self.__find_recipient_by_sharing_id(recipient_sharing_identifier)
      if existing != None and existing != recipient:
        raise Exception(f"cannot add recipient {recipient} because another recipient {existing} alread defined using the same recipient_shareing_identifier={recipient_sharing_identifier}. re-run again, however, use {existing} as recipient instead of {recipient}. Alternatively, drop {existing} first so it can be created.")
      display(self.__spark_sql(f'create recipient if not exists {recipient} using ID "{recipient_sharing_identifier}";'))
      self.__log(f'databirkcs recipient {recipient} created using the sharing identifier provided. inform the recipient so they can start reading the shares.')
      self.__spark_sql("select concat('DeltaShareRecipient(provider_shareing_identifier=\"', current_metastore(), '\", catalog=\"hive_metastore\").create_remotely_linked_tables(share=\"', '{self.share}', '\")') as run_this_code_at_recipient")

    self.__spark_sql(f'GRANT SELECT on SHARE {self.share} TO RECIPIENT {recipient};')
    self.__log(f'recipient {recipient} granted SELECT on share {self.share}')
    return self
  
  def __find_recipient_by_sharing_id(self, recipient_sharing_identifier:str)->str:
    recipients = list(self.__spark_sql("show recipients;").where("authentication_type='DATABRICKS'").select("recipient").toPandas()['recipient'])
    recipient_name = None
    for recipient in recipients:
      df = self.__spark_sql(f"desc recipient {recipient};")
      cloud = ""
      region=""
      metastore_id = ""
      for row in df.collect():
        if row.info_name == "recipient_name":
          recipient_name = row.info_value
        if row.info_name == "cloud":
          cloud = row.info_value
        if row.info_name == "region":
          region = row.info_value
        if row.info_name == "metastore_id":
          metastore_id = row.info_value
      if f"{cloud}:{region}:{metastore_id}" == recipient_sharing_identifier:
        break
    return recipient_name
  
  def remove_recipient(self, recipient:str):
    """
    Removes SELECT access to the specified recipient from the share.
    
    Args:
        recipient (str): Name of the recipient.
    
    Returns:
        self
    """
    try:
      self.__spark_sql(f'REVOKE SELECT ON SHARE {self.share} FROM RECIPIENT {recipient};')
      self.__log(f'SELECT access on share {self.share} is revoked from  recipient {recipient}')
    except Exception as e:
      self.__log("exception occured, message is: " + str(e))
    return self
  
  def drop_recipient(self, recipient:str):
    """
    Drops the specified recipient.
    
    Args:
        recipient (str): Name of the recipient.
    
    Returns:
        self
    """
    try:
      self.__log(f"recipient {recipient} will be dropped.")
      self.__spark_sql(f'DROP RECIPIENT IF EXISTS {recipient};')
    except Exception as e:
      self.__log("exception occured, message is: " + str(e))
    return self
    
  def __spark_sql(self, sql):
    print(sql)
    print()
    return spark.sql(sql)

  def __log(self, thing):
    print("[info] " + thing)
    pass
  
  def __get_database_objects(self, object_type, source, selector):
    return list(self.__spark_sql(f"show {object_type} in {source};").toPandas()[selector])